In [20]:
import datalab.bigquery as bq

In [21]:
%%sql --module logspreview1
SELECT response_text FROM [shwetaproject-001:eMee_SMCC.tweets] where not response_text in ("null", "NULL")

In [22]:
import datalab.bigquery as bq
# Create a query using the SQL module defined above.
q = bq.Query(logspreview1)
# Run the query, with caching turned off (for sample purposes only), so we're sure to be
# able to retrieve metadata, such as bytes processed from the resulting query job.
results = q.results()
list1=list(results)

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

tweets=[]

In [25]:
import operator 
import json
from collections import Counter
from numpy import *
 
from nltk.corpus import stopwords
from nltk import bigrams 
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

count_all = Counter()
index=0
for d in list1:
    # Create a list with all the terms
    temp=dict();
    tweet = [term for term in preprocess(d['response_text']) 
              if term not in stop ] 
    # Update the counter
    terms_bigram = bigrams(tweet)
    #print terms_bigram
    temp['tweet']=tweet
    temp['index']=index
    index=index+1
    #print temp
    tweets.append(temp)
    count_all.update(terms_bigram)
    # Print the first 5 most frequent words
    #print 
keys=['sabtv','BBCEarth','SonyTV','SonyPIX','SonySIX']
print(count_all.most_common(5))
#for k in tweets:
  #print k

[((u'SonySIX', u'Ans'), 6), ((u'Ans', u'7'), 5), ((u'7', u'ACTING'), 3), ((u'SINGING', u'TNAWeekendTakeover'), 3), ((u'MODELLING', u'amp'), 3)]


In [26]:
import numpy as np
import tensorflow as tf

num_vectors = 1000
num_clusters = 2
num_steps = 100

vector_values = []
for i in tweets:
  vector_values.append([i['tweet'],i['tweet'].count(keys[0]),i['tweet'].count(keys[1]),i['tweet'].count(keys[2]),i['tweet'].count(keys[3]),i['tweet'].count(keys[4]),i['index']])
#print vector_values

In [27]:
#calculating centroids 5
centroids=[]
i=1
while i<6:
  for j in vector_values:
    if j[i]==1:
      centroids.append(j)
      #print j
      break
  i=i+1
#print centroids

In [28]:
i=0
clusters=[]
clusters.append([])
clusters.append([])
clusters.append([])
clusters.append([])
clusters.append([])
sess=tf.Session()
for d in vector_values:
  x=[]
  #x.append(d[0])
  for j in centroids:
    u1 = tf.constant(float(d[1]), name='u1')
    v1 = tf.constant(float(d[2]), name='v1')
    w1 = tf.constant(float(d[3]), name='w1')
    x1 = tf.constant(float(d[4]), name='x1')
    y1 = tf.constant(float(d[5]), name='y1')
    u2 = tf.constant(float(j[1]), name='u2')
    v2 = tf.constant(float(j[2]), name='v2')
    w2 = tf.constant(float(j[3]), name='w2')
    x2 = tf.constant(float(j[4]), name='x2')
    y2 = tf.constant(float(j[5]), name='y2')
    model = tf.initialize_all_variables()
    with sess.as_default():
      x.append(tf.sqrt(tf.pow(tf.sub(u1,u2),2)+tf.pow(tf.sub(v1,v2),2)+tf.pow(tf.sub(w1,w2),2)+tf.pow(tf.sub(x1,x2),2)+tf.pow(tf.sub(y1,y2),2)).eval())
  clusters[x.index(min(x))].append(list1[d[6]]['response_text'])
print "cluster 1 :subtv"
for k in clusters[0]:
  print k
print "\ncluster 2 :BBCEarth"
for k in clusters[1]:
  print k
print "\ncluster 3 :SonyTV"
for k in clusters[2]:
  print k
print "\ncluster 4 :SonyPIX"
for k in clusters[3]:
  print k
print "\ncluster 5 :SonySIX"
for k in clusters[4]:
  print k

cluster 1 :subtv
.@sabtv Woah they are Looking super mind-blowing #SABKiDiwaliExclusives
@sabtv one is ekmev secretary Bhide other one I cant get #SABKiDiwaliExclusives
Looking forward to watching this show - #DilDekeDekhoOnSAB @sabtv  https://t.co/HbbvhL7ubI
@pritambakshi501 this show will start from 18 October at night 10pm @sabtv #DilDekeDekhoOnSAB
#DilDekeDekhoOnSAB teaser is really superb. Watching it on loop @sabtv https://t.co/TVM2muHNao
@sabtv Their JODI looks so cool onstage. Love this couple of Jetha and Daya. #SABKiDiwaliExclusives @sabtv
Dil Deke Dekho Press Conference talks animatedly at Press Conference #DilDekeDekhoOnSAB https://t.co/xMXjGggFrt
The show will make you wordless to praise it kyunki umar nahin dekhte jab ho jaata hai pyaar! #DilDekeDekhoOnSAB @sabtv
#DilDekeDekhoOnSABBuddies The Stage is all Set for  Press Conference!Waiting for more details from @sabtv  https://t.co/8iIFHYFbYz
The star cast of #DilDekeDekhoOnSAB are interacting with the media at the grand p